# The MUSIC algorithm

The MUSIC algorithm is used to estimate the direction of arrival of multiple signals based on received signal on a sensor array.

> Schmidt, R. (1986). Multiple emitter location and signal parameter estimation. IEEE transactions on antennas and propagation, 34(3), 276-280.

It has been used in EEG recordings as well as extracellular tetrode recordings. For tetrode recordings, the location is generally not that important as identities of cells are determined using spike sorting. However, the estimated location of a cell could be a useful dimensionality reduction method.

> Lee, C. W., Szymanska, A. A., Wu, S. C., Swindlehurst, A. L., & Nenadic, Z. (2013). A Method for Neuronal Source Identification. arXiv preprint arXiv:1311.3952.



In [10]:
import numpy as np
import plotly.express as px

## Step 1: Create a simulated tetrode?

Suppose our tetrode is cut in a plane. Electrodes are approximately 50 micrometers apart [[1][1]].

[1]: https://doi.org/10.1109/IEMBS.2011.6089898 "Lee, C. W., King, C. E., Wu, S. C., Swindlehurst, A. L., & Nenadic, Z. (2011). Signal source localization with tetrodes: Experimental verification. In 2011 Annual International Conference of the IEEE Engineering in Medicine and Biology Society (pp. 67-70). IEEE."

In [29]:
locations = np.array([[0, 0, 0],
                      [50, 0, 0],
                      [50, 50, 0],
                      [0, 50, 0]])

In [31]:
px.scatter_3d(x=locations[:,0], y=locations[:,1], z=locations[:,2])

### An equation modeling voltage decay over distance

In basically all of this, we assume a point source of voltage with a scalar amplitude varying with time. Thus, we are interested in calculating the distance between a sensor and the point source. We have an equation for the decay of voltage over distance.

$$a_i = \frac{1}{4\pi \sigma r_i(\textbf{r})}$$

The $a_i$ is almost like a scaling factor multiplied to the point source's raw amplitude to calculate how much the signal is modulated by the distance. This hides some of the complexity involved in calculating the Euclidean distance though.

### Looking at eigenvectors in numpy

In [33]:
from numpy import linalg as LA

A = np.array([[3, 2],
              [0, 2]]) 
lam, u = LA.eig(A)

### Reading MDA files for the raw stuff
I'm very sorry, but these files are locally on my computer. It's not reproducible. It's a really big file too.

In [21]:
import pyms.mlpy as mlpy
import pandas as pd

In [4]:
X=mlpy.readmda('/run/media/trevortknguyen/a8a15e97-b12c-4825-8482-0be8f89aac96/20170920_remy_02_r1/20170920_remy_02_r1.mda/20170920_remy_02_r1.nt1.mda')

In [6]:
X.shape

(4, 36975610)

There's 20 minutes of 30kHz samples.

In [9]:
X.shape[1] / 30000 / 60

20.542005555555555

### Trying to do MUSIC?

I am not sure if I need to get an excerpt of the

In [50]:
df = pd.DataFrame()
samples = 10000
signal = np.concatenate([
    X[0, :samples],
    X[1, :samples],
    X[2, :samples],
    X[3, :samples],
])

label = np.concatenate([
    np.repeat('a',samples),
    np.repeat('b',samples),
    np.repeat('c',samples),
    np.repeat('d',samples),
])

xs = np.concatenate([
    np.linspace(0, samples, samples),
    np.linspace(0, samples, samples),
    np.linspace(0, samples, samples),
    np.linspace(0, samples, samples),
])

In [54]:
px.line(x=xs, y=signal, color=label)

In [58]:
u, s, vh = np.linalg.svd(X[:,:10000], full_matrices=True)

### Trying to understand some random GitHub code



In [70]:
X = np.load('recieved_signal_data.npy')

In [71]:
X.shape

(10, 100)

In [72]:
X[0,:10]

array([ 3.37693408+1.81081960e+00j,  0.13366669+1.22637111e+00j,
       -0.48003664+2.29546724e+00j, -0.49502534+3.25077542e-02j,
       -3.87467822+1.74999434e-15j, -0.06791149+4.72040194e-01j,
        0.75928956+2.65097189e+00j, -2.26817243+2.14792601e+00j,
        1.73612622-1.34568200e+00j,  1.46346129-3.04292958e-15j])

In [81]:
X.reshape((10*100,)).shape

(1000,)

In [89]:
signals = X.reshape((10*100,))

In [93]:
labels = np.repeat(['a','b','c','d', 'e', 'f', 'g', 'h', 'i', 'j'], 100)

In [91]:
times = np.tile(np.arange(100), 10)

In [102]:
px.line(x=times, y = signals.imag, color=labels)

In [103]:
px.line(x=times, y = signals.real, color=labels)